In [39]:
from eth_utils import address
from web3 import Web3
import os
from solcx import compile_standard, install_solc
import json

In [40]:
version = '0.8.0'
install_solc(version)

<Version('0.8.0')>

In [86]:
smart_contract = "EnergyTrading.sol"

with open(smart_contract, "r") as file:
    contract_file = file.read()

compiled_sol = compile_standard(
    {
        "language": "Solidity",
        "sources": {"EnergyTrading.sol": {"content": contract_file}},
        "settings": {
            "outputSelection": {
                "*": {
                    "*": ["abi", "metadata", "evm.bytecode", "evm.bytecode.sourceMap"]
                }
            }
        },
    },
    solc_version=version,  # type: ignore
)

with open("compiled_code.json", "w") as file:
    json.dump(compiled_sol, file)

In [87]:
# get bytecode
bytecode = compiled_sol["contracts"][smart_contract]["EnergyTrading"]["evm"]["bytecode"]["object"]

# get abi
abi = json.loads(compiled_sol["contracts"][smart_contract]["EnergyTrading"]["metadata"])["output"]["abi"]

In [88]:
# set up connection
w3 = Web3(Web3.HTTPProvider("HTTP://172.21.128.1:7545"))
chain_id = 1337
owner_address = "0x952b1253dF851D217501Fa66fa5f4a8cC9025Eac"
private_key = "0c1aa4f476b3a2768675e3d1190ba26df71fd2e3044e01712db59538cdb82760"

In [89]:
# initialize contract
EnergyTrading = w3.eth.contract(abi=abi, bytecode=bytecode)
nonce = w3.eth.get_transaction_count(owner_address)

In [90]:
# set up transaction from constructor which executes when firstly
transaction = EnergyTrading.constructor().build_transaction(
    {
        "gasPrice": w3.eth.gas_price,
        "chainId": chain_id, 
        "from": owner_address, 
        "nonce": nonce
    }
)

signed_tx = w3.eth.account.sign_transaction(transaction, private_key=private_key)
tx_hash = w3.eth.send_raw_transaction(signed_tx.raw_transaction)
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)

In [91]:
with open("contract_abi.json", "w") as file:
    json.dump(abi, file)

In [92]:
energy_trading_sol = w3.eth.contract(abi=abi, address=tx_receipt.contractAddress)

payer_address = "0xB28d5f7929AA1F5f6d5811C692d59255ABFdD530"
private_key = "90c2213fbee613082e2340ed07de21e526d5b4a9b04dbc236780e799ee6bfa98"
nonce = w3.eth.get_transaction_count(payer_address)
value = 4

call_fun = energy_trading_sol.functions.registerUser().build_transaction(
    {
        "gasPrice": w3.eth.gas_price,
        "chainId": chain_id, 
        "from": payer_address,
        "nonce": nonce
    }
)

sign_call_fun = w3.eth.account.sign_transaction(call_fun, private_key=private_key)
tx_call_fun_hash = w3.eth.send_raw_transaction(sign_call_fun.raw_transaction)
tx_call_fun_receipt = w3.eth.wait_for_transaction_receipt(tx_call_fun_hash)

In [101]:
# dump contract address to file
with open("../backend/utils/contract_address.json", "w") as file:
    json.dump({"contractAddress": tx_receipt.contractAddress}, file)

# dump contract abi to file
with open("../backend/utils/contract_abi.json", "w") as file:
    json.dump(abi, file)

In [95]:
energy_trading_sol = w3.eth.contract(abi=abi, address=tx_receipt.contractAddress)

payer_address = "0xB28d5f7929AA1F5f6d5811C692d59255ABFdD530"
private_key = "90c2213fbee613082e2340ed07de21e526d5b4a9b04dbc236780e799ee6bfa98"
nonce = w3.eth.get_transaction_count(payer_address)
value = 4

call_fun = energy_trading_sol.functions.offerEnergy(
    15000, 15, 120
).build_transaction(
    {
        "gasPrice": w3.eth.gas_price,
        "chainId": chain_id, 
        "from": payer_address,
        "nonce": nonce
    }
)

sign_call_fun = w3.eth.account.sign_transaction(call_fun, private_key=private_key)
tx_call_fun_hash = w3.eth.send_raw_transaction(sign_call_fun.raw_transaction)
tx_call_fun_receipt = w3.eth.wait_for_transaction_receipt(tx_call_fun_hash)

print(tx_call_fun_receipt)

AttributeDict({'transactionHash': HexBytes('0x786a97a34c4f4c47530fb20b44a207b666ead356a062c1c73d63588a117506ff'), 'transactionIndex': 0, 'blockHash': HexBytes('0xc518f527d30dfeb44bc23e24ea2105dabd7ec2f6b2e4874a020ec2d3afef606a'), 'blockNumber': 29, 'from': '0xB28d5f7929AA1F5f6d5811C692d59255ABFdD530', 'to': '0x7f249b0565c8103Cc5796D77d4D101c3dAA4cc72', 'gasUsed': 138892, 'cumulativeGasUsed': 138892, 'contractAddress': None, 'logs': [AttributeDict({'logIndex': 0, 'transactionIndex': 0, 'transactionHash': HexBytes('0x786a97a34c4f4c47530fb20b44a207b666ead356a062c1c73d63588a117506ff'), 'blockHash': HexBytes('0xc518f527d30dfeb44bc23e24ea2105dabd7ec2f6b2e4874a020ec2d3afef606a'), 'blockNumber': 29, 'address': '0x7f249b0565c8103Cc5796D77d4D101c3dAA4cc72', 'data': HexBytes('0x0000000000000000000000000000000000000000000000000000000000000003000000000000000000000000b28d5f7929aa1f5f6d5811c692d59255abfdd5300000000000000000000000000000000000000000000000000000000000003a98000000000000000000000000000000

In [ ]:
energy_trading_sol.events.EnergyOffered().process_receipt(tx_call_fun_receipt)

<Event EnergyOffered(address,uint256,uint256)>

In [96]:
energy_trading_sol = w3.eth.contract(abi=abi, address=tx_receipt.contractAddress)

payer_address = "0xB28d5f7929AA1F5f6d5811C692d59255ABFdD530"
private_key = "90c2213fbee613082e2340ed07de21e526d5b4a9b04dbc236780e799ee6bfa98"
nonce = w3.eth.get_transaction_count(payer_address)
value = 4

# Call the function and get the return value
active_offers = energy_trading_sol.functions.getActiveOffers().call({
    "from": payer_address
})

active_offers

# call_fun = energy_trading_sol.functions.getActiveOffers().build_transaction(
#     {
#         "gasPrice": w3.eth.gas_price,
#         "chainId": chain_id, 
#         "from": payer_address,
#         "nonce": nonce
#     }
# )

# sign_call_fun = w3.eth.account.sign_transaction(call_fun, private_key=private_key)
# tx_call_fun_hash = w3.eth.send_raw_transaction(sign_call_fun.raw_transaction)
# tx_call_fun_receipt = w3.eth.wait_for_transaction_receipt(tx_call_fun_hash)

[(0,
  '0xB28d5f7929AA1F5f6d5811C692d59255ABFdD530',
  20000,
  50,
  1732189794,
  '0x0000000000000000000000000000000000000000',
  0,
  False),
 (1,
  '0xB28d5f7929AA1F5f6d5811C692d59255ABFdD530',
  10000,
  10,
  1732189799,
  '0x0000000000000000000000000000000000000000',
  0,
  False),
 (2,
  '0xB28d5f7929AA1F5f6d5811C692d59255ABFdD530',
  15000,
  15,
  1732193407,
  '0x0000000000000000000000000000000000000000',
  0,
  False)]

In [77]:
active_offers

[0, 1, 2]

In [64]:
w3.eth.get_balance(tx_receipt.contractAddress) / 1e18

0.0